In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import h5py
import pandas as pd


## Set up for the Project
#### Code to load files, obtain needed parameters etc

In [ ]:
##Define files
filename0="3D_temperature_9_dist_bins_nside_128.hdf5"
filename1="3D_temperature_17_dist_bins_nside_128.hdf5"


In [ ]:
##Functions to load and plot temperature map
def load_3D_temperature_data(file):
    '''
    A function that loads the 3D dust temperature map and returns a dictionary of the data. This dictionary contains the temperature, distance slices, the number of distance bins, the HEALPix resolution Nside, and the healpix ordering (nesting). 
    Parameters: 
        file------------------------------hdf5 dust temperature file
    Returns:
        data_dict-------------------------dictionary

    '''
    data_dict = {}
    with h5py.File(file, "r") as g:    
        data_dict["temperatures"]=g["temperature"][()] # this is (healpix x distance_bin)
        data_dict["distance_slices"] = g["distance_slices"][()]
        data_dict["nside"] = g.attrs["nside"]
        data_dict["nr_of_distance_bins"] = g.attrs["nr_of_distance_bins"]
        data_dict["healpix_ordering"] = g.attrs["healpix_ordering"] #nest
        g.close()
    return data_dict

##Function to load positions and distance of the different star formation tracers
def load_sftracer(tracer_data):
    '''
    A function that loads the longitude, latitude and distance of a star formation tracer. It converts them from degrees to radians, and converts the latitude into colatitude. If no distance is found, the function only returns theta and phi. 

    Parameters: 
        tracer_data-------------------------------pandas dataframe, must have a component 'l' and a component 'b', optionally can have 'D'

    Returns:
        theta-------------------------------------array-like, colatitude in radians
        phi---------------------------------------array-like, longitude in radians
        distance----------------------------------array-like, distance in kpc
    '''
    if 'D' in tracer_data: #If the file has distance create a distance variable and return it
        distance = tracer_data['D'] 
        phi = np.radians(tracer_data['l'])
        theta = np.radians(90. - tracer_data['b']) #Change from latitude to colatitude

        return theta, phi, distance
    else: #If there is no distance only return that and phi
        phi = np.radians(tracer_data['l'])
        theta = np.radians(90. - tracer_data['b'])
        print("No distance measurement")
    return theta, phi


##Function to assign each position to one of the distance slices
def assign_distance_slice(data_dict, theta, phi, distance):
    '''
    A function that assigns one of the distance slices from the 3D dust temperature map, to the position of each object. Based on the distance it checks to see in which slice it belongs and then makes a new array of the positions with their assigned distance slice. 

    Parameters:
        data_dict-------------------------------dictionary, dictionary of one of the 3D dust temperature maps
        theta-----------------------------------array-like, colatitude of the object in radians
        phi-------------------------------------array-like, longitude of an object in radians
        distance--------------------------------array-like, distance to object in kpc

    Returns:
        theta_slice-----------------------------array-like, colatitude that now has an assigned distance slice
        phi-------------------------------------array-like, longitude that now has an assigned distance slice
        
    '''
    
    distance_bins = data_dict["nr_of_distance_bins"]
    distance_slices = data_dict["distance_slices"]
    theta_slice = [[] for i in range(distance_bins)] #Make empty array to store positions with distance slices
    phi_slice = [[] for i in range(distance_bins)]

    for idx, dist in enumerate(distance): #Do this for each distance value
        for ds_idx in range(distance_bins): #Check to see which distance slice sf distance is
            if ds_idx ==0 and dist < distance_slices[0]: #Condition for first distance slice
               theta_slice[0].append(theta[0])
               phi_slice[0].append(phi[0])
                  
            elif ds_idx > 0 and distance_slices[ds_idx-1] <= dist <= distance_slices[ds_idx]: #Condition for all other distance slices
                theta_slice[ds_idx].append(theta[idx])
                phi_slice[ds_idx].append(phi[idx])
                
    return theta_slice, phi_slice #Return "sliced" values
            

def plot_3D_temperature_slice_maps(data_dict, theta, phi, theta2, phi2):
    '''
    A function make a healpix map of the 3D dust temperature maps at each distance slices. It overplots a scatter plot of different star formation tracers

    Parameters:
        data_dict-------------------------------dictionary, dictionary of one of the 3D dust temperature maps
        theta-----------------------------------array-like, colatitude of a YSO in radians
        phi-------------------------------------array-like, longitude of a YSO in radians
        theta2-----------------------------------array-like, colatitude of a SFC in radians
        phi2-------------------------------------array-like, longitude of a SFC in radians

    Returns:
        A healpix map at each distance slice with two scatter plots overlaid. 
    '''
    Ts = data_dict["temperatures"]
    model_nslices = data_dict["nr_of_distance_bins"]
    model_dist_slices = data_dict["distance_slices"]
    for ds_index in range(model_nslices):                                 
        hp.mollview(Ts[:,ds_index],title=r"$T$ at distance slice "+str(ds_index) +\
                                   " at "+'{:.2f}'.format(model_dist_slices[ds_index])+" kpc",nest=True,min=10,max=25, unit='K')
        hp.projscatter(theta2[ds_index], phi2[ds_index], marker='*', s=100, color='red', alpha=0.4) #Added plotting positions of sf tracer
        hp.projscatter(theta[ds_index], phi[ds_index], s = 100, marker='o', alpha = 0.9, color='blue') #Added plotting positions of sf tracer
        plt.legend(['YSOs', 'SFCs'])
        plt.savefig('tempmap_{}.pdf'.format(ds_index))
        


In [ ]:
##Load temperature map

#data_dict0=load_3D_temperature_data(filename0) #Not needed for this project but likely in the future
data_dict1=load_3D_temperature_data(filename1)

In [ ]:
##Define Nside and Npix
Nside = 128 ##based on dust map
Npix = hp.nside2npix(Nside)

In [ ]:
##Get positions of the YSOs

#This is for all YSOs in the catalog
YSOs = pd.read_csv("SPICY_YSOs_new.csv")
theta_ysos, phi_ysos = load_sftracer(YSOs)

#Only the YSOs with distance
YSOs_with_dist = pd.read_csv("YSOS_withdist.csv")
theta_ysos_wdist, phi_ysos_wdist, dist_ysos = load_sftracer(YSOs_with_dist)

#Get distances slices for YSOs
theta_ysos_sliced, phi_ysos_sliced = assign_distance_slice(data_dict1, theta_ysos_wdist, phi_ysos_wdist, dist_ysos)

In [ ]:
##Get positions of the SFCs
linetoskip = [5, 7, 9, 12, 15, 23, 35, 36, 39] ##skipping lines that have two distances

SFCs = pd.read_csv("SFC_Regions.csv", skiprows = linetoskip) ##read file, skipping lines

theta_sfc, phi_sfc, dist_sfc = load_sftracer(SFCs) ##get latitude, longitude and distance

theta_sfc_sliced, phi_sfc_sliced = assign_distance_slice(data_dict1, theta_sfc, phi_sfc, dist_sfc) ## get positions at different distances

# Part 1 
#### Plot the positions (longitude and latitude) of all the YSOs and SFCs 

In [ ]:
##Plot star formation tracers in sky on their own (not sliced yet)
hp.mollview(title = 'Star Formation Tracers') #Create empty map
hp.projscatter(theta_ysos, phi_ysos, marker = '*', s=10, alpha=0.01, color='royalblue')
hp.projscatter(theta_sfc, phi_sfc, marker = 'o', facecolors='none', s=50, alpha=1, color='red')
plt.savefig('Tracers_all.pdf')
plt.show()
plt.close()

# Part 2 
#### Plot the positions of the YSOs and SFCs at each distance slice in Ioana's dust temperature map

In [ ]:
##Plot temperature map
plot_3D_temperature_slice_maps(data_dict1, theta_sfc_sliced, phi_sfc_sliced, theta_ysos_sliced, phi_ysos_sliced)